# 建构SQL查询助手

在`github`上看到[`SQLBot`](https://github.com/dataease/SQLBot)这个项目，这个代码比较复杂，于是想是否可以使用`langchain`来实现，发现了[Build a Question/Answering system over SQL data](https://python.langchain.com/docs/tutorials/sql_qa/)这个教程。

从高层次来看，这些系统的步骤如下：

- 将问题转换为 `SQL` 查询：模型将用户输入转换为 `SQL` 查询。
- 执行 `SQL` 查询：执行查询。
- 回答问题：模型使用查询结果响应用户输入。

同理，为说明简单，这里也是使用示范中的 `SQLite` 测试数据库文件 `Chinook.db`。

可以使用 `SQLAlchemy` 驱动的 `SQLDatabase` 类与它交互

In [8]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
tables = list(db.get_usable_table_names())
print(tables)
# db.run(f'select * from sqlite_master where type="table" and name="{tables[0]}";')
db.run(f'PRAGMA table_info("{tables[0]}");')

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(0, 'AlbumId', 'INTEGER', 1, None, 1), (1, 'Title', 'NVARCHAR(160)', 1, None, 0), (2, 'ArtistId', 'INTEGER', 1, None, 0)]"

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain.chat_models import init_chat_model

llm = init_chat_model("deepseek-chat", model_provider="deepseek")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_message = """
给定一个输入问题，创建一个语法正确的 {dialect} 查询来运行以帮助找到答案。
除非用户在其问题中指定了他们希望获得的特定示例数量，否则始终将查询限制为最多 {top_k} 个结果。
你可以按相关列对结果进行排序，以返回数据库中最有趣的示例。切勿查询特定表中的所有列，只请求与问题相关的少数列。
注意仅使用你在模式描述中能看到的列名。注意不要查询不存在的列。同时，注意哪一列在哪个表中。
仅使用以下表:
{table_info}
"""

user_prompt = "问题: {input}"

query_prompt_template = ChatPromptTemplate(
    [("system", system_message), ("user", user_prompt)]
)

for message in query_prompt_template.messages:
    message.pretty_print()